In [131]:
# VER TODO-LIST em README.md
import json
import urllib.request
from toolset import (get_examples, get_frequency,
                    get_text, get_tokens)
from cltk import NLP
from cltk.lemmatize.grc import GreekBackoffLemmatizer
lemmatizer = GreekBackoffLemmatizer()

from cltk.alphabet.text_normalization import cltk_normalize

In [8]:
creed, wf = get_text("creed.txt")
tokens, token_set, phrases = get_tokens(creed)
frequency = get_frequency(tokens)
cltk_nlp = NLP(language="grc")
cltk_doc = cltk_nlp.analyze(creed)

wordlist = []
for pair in frequency:
    token = pair[0]
    wordlist.append(token)

‎𐤀 CLTK version '1.0.24'.
Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`, `GreekNERProcess`.


In [10]:
def request(action, **params):
    return {'action': action,
            'params': params,
            'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))

    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')

    if 'error' not in response:
        raise Exception('response is missing required error field')
    
    if 'result' not in response:
        raise Exception('response is missing required result field')

    if response['error'] is not None:
        raise Exception(response['error'])

    return response['result']

noteIDs = invoke('findNotes', query='deck:Liturgia')
notesInfo = invoke('notesInfo', notes=noteIDs)

In [37]:
notes = []
for card in notesInfo:
    notes.append(cltk_normalize(card['fields']['Word']['value']))
fwl = [cltk_normalize(word) for word in wordlist if word not in notes]
notes_group = lemmatizer.lemmatize(notes)
fwl_group = lemmatizer.lemmatize(fwl)

In [51]:
notes_lemmas = set([tuple[1] for tuple in notes_group])
fwl_lemmas = set([tuple[1] for tuple in fwl_group])

In [130]:
filtered_fwl_lemmas = [word for word in fwl_lemmas if word not in notes_lemmas]
reworked_fwl = [tuple for tuple in fwl_group if tuple[1] in filtered_fwl_lemmas]
fwl_lemmas

{'Γραφάς',
 'Εἰς',
 'Θεοῦ',
 'Θεόν',
 'Θεὸν',
 'Κύριον',
 'Μαρίας',
 'Παρθένου',
 'Πατέρα',
 'Πατρί',
 'Πατρὶ',
 'Πατρὸς',
 'Πιλάτου',
 'Πιστεύω',
 'Πνεύματος',
 'Πνεῦμα',
 'Ποντίου',
 'Προσδοκῶ',
 'Προφητῶν',
 'Σταυρωθέντα',
 'Τὸν',
 'Υἱὸν',
 'Υἱῷ',
 'Φῶς',
 'Χριστόν',
 'αἰών',
 'βάπτισμα',
 'βασιλεία',
 'γίγνομαι',
 'γεννάω',
 'δεξιός',
 'δι ̓',
 'δόξα',
 'εἰμί',
 'εἷς',
 'ζάω',
 'ζωή',
 'ζωοποιός',
 'θάπτω',
 'καί',
 'καθολικὴν',
 'κατά',
 'κατάἕζομαι',
 'κατέρχομαι',
 'κρίνω',
 'κύριος',
 'λαλέω',
 'μέλλω',
 'μετά',
 'μονογενής',
 'νεκρός',
 'οὐρανός',
 'οὐ',
 'οὐρανοὺς',
 'οὐρανός',
 'πάλιν',
 'πάσχω',
 'παντοκράτωρ',
 'ποιέω',
 'ποιητής',
 'πρό',
 'πᾶς',
 'σαρκόω',
 'συνδοξάζω',
 'σωτηρία',
 'σύν',
 'σύν,πρόσκυνάω',
 'τέλος',
 'τε',
 'τρίτος',
 'φώς',
 'ἀληθινός',
 'ἀνάστασις',
 'ἀνέρχομαι',
 'ἀνίστημι',
 'ἀποστολικὴν',
 'ἀόρατος',
 'ἁμαρτία',
 'ἄνθρωπος',
 'ἄφεσις',
 'ἅγιος',
 'Ἀμήν',
 'Ἁγίου',
 'Ἅγιον',
 'ἐκ',
 'ἐκπορεύω',
 'ἐνανθρωπέω',
 'ἔρχομαι',
 'Ἐκκλησίαν',
 'ἡμέτερος',


In [60]:
import pandas as pd

df = pd.DataFrame(reworked_fwl)
df.to_csv("output.csv", encoding="utf-8", header=None)


In [123]:
new_df = pd.read_csv("filtered.csv", header=None, index_col=0)
repeats = []
for item in new_df.iterrows():
    if item[0] == item[1][1]:
        repeats.append(item[0])

In [124]:
final_list = []
for item in new_df.iterrows():
    if item[0] in repeats:
        final_list.append(item[0])
    else:
        final_list.append((item[0], item[1][1]))

In [128]:
final_list

['Πιστεύω',
 ('παντοκράτορα', 'παντοκράτωρ'),
 ('ποιητὴν', 'ποιητής'),
 ('ὁρατῶν', 'ὁρατός'),
 ('ἀοράτων', 'ἀόρατος'),
 ('μονογενῆ', 'μονογενής'),
 ('ἀληθινὸν', 'ἀληθινός'),
 'οὐ',
 ('ὁμοούσιον', 'ὁμοούσιος'),
 ('πάντα', 'πᾶς'),
 ('ἡμετέραν', 'ἡμέτερος'),
 ('σωτηρίαν', 'σωτηρία'),
 ('κατελθόντα', 'κατέρχομαι'),
 ('σαρκωθέντα', 'σαρκόω'),
 'Παρθένου',
 ('ἐνανθρωπήσαντα', 'ἐνανθρωπέω'),
 'Σταυρωθέντα',
 ('ὑπὲρ', 'ὑπέρ'),
 'Ποντίου',
 'Πιλάτου',
 ('παθόντα', 'πάσχω'),
 ('ταφέντα', 'θάπτω'),
 ('ἀναστάντα', 'ἀνίστημι'),
 ('τρίτῃ', 'τρίτος'),
 'Γραφάς',
 ('ἀνελθόντα', 'ἀνέρχομαι'),
 'οὐρανοὺς',
 ('καθεζόμενον', 'κατάἕζομαι'),
 'πάλιν',
 ('μετὰ', 'μετά'),
 ('κρῖναι', 'κρίνω'),
 ('ζῶντας', 'ζάω'),
 ('νεκρούς', 'νεκρός'),
 ('βασιλείας', 'βασιλεία'),
 ('οὐκ', 'οὐ'),
 'τέλος',
 ('ζωοποιόν', 'ζωοποιός'),
 ('ἐκπορευόμενον', 'ἐκπορεύω'),
 ('σὺν', 'σύν'),
 ('συμπροσκυνούμενον', 'σύν,πρόσκυνάω'),
 ('συνδοξαζόμενον', 'συνδοξάζω'),
 ('λαλῆσαν', 'λαλέω'),
 'Προφητῶν',
 ('μίαν', 'εἷς'),
 'καθολικὴν',
 'ἀπ